In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/my-data-movie/reviews_11_10.csv


In [ ]:
from google.colab import drive
import pandas as pd
import os

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from datasets import Dataset
import evaluate
import numpy as np

from transformers import pipeline
from sklearn.metrics import classification_report
from transformers import TrainingArguments
from transformers import Trainer

from transformers import BertForSequenceClassification
from transformers import BertTokenizer, TFBertModel
from transformers import DataCollatorWithPadding

In [ ]:
# !pip install wandb
!pip uninstall -y wandb
os.environ["WANDB_DISABLED"] = "true"

!pip install evaluate

## Importation des données

In [ ]:
df = pd.read_csv('/kaggle/input/big-dataset/reviews.csv')
df = df.rename(columns={'review': 'text', 'label': 'labels'})
print(df.shape)
df.head()

(48297, 2)


,text,labels
0,Now 'This' is a Funny thrill ride of a movie t...,positif
1,A real good miniseries I just saw this mini-se...,positif
2,Fun Movie A movie doesn't need lots of hi-tech...,positif
3,good film but something is lacking in the chem...,positif
4,great drama movie Philadelphia is one of the b...,positif


In [ ]:
df['labels'].value_counts()

labels
positif    25000
négatif    23297
Name: count, dtype: int64

In [ ]:
# import re
# import nltk
# from nltk.corpus import stopwords

# nltk.download("stopwords")
# stop_words = set(stopwords.words("english"))

# def clean_text(text):
#     text = text.lower()
#     text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
#     # text = ' '.join(word for word in text.split() if word not in stop_words)
#     return text
# df["text"] = df["review"].apply(clean_text)

In [ ]:
#Label encoding
from sklearn import preprocessing
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(df['labels'].tolist())
df['labels'] = le.transform(df['labels'].tolist())
df.head()

,text,labels
0,Now 'This' is a Funny thrill ride of a movie t...,1
1,A real good miniseries I just saw this mini-se...,1
2,Fun Movie A movie doesn't need lots of hi-tech...,1
3,good film but something is lacking in the chem...,1
4,great drama movie Philadelphia is one of the b...,1


In [ ]:
df_train, df_test = train_test_split(df,test_size=0.2)
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)
print(train_dataset, test_dataset)

Dataset({
    features: ['text', 'labels', '__index_level_0__'],
    num_rows: 38637
}) Dataset({
    features: ['text', 'labels', '__index_level_0__'],
    num_rows: 9660
})


## bert-base-uncased

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def tokenize_function(example):
    return tokenizer(example['text'], truncation=True, padding='max_length')

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)



Map:   0%|          | 0/38637 [00:00<?, ? examples/s]

Map:   0%|          | 0/9660 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


# tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_train.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    #report_to = None
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics

)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer.train()

In [ ]:
#Evaluation du model

from sklearn.metrics import classification_report

preds = trainer.predict(tokenized_test)
preds = np.argmax(preds[:3][0],axis=1)
GT = df_test['labels'].tolist()
print(classification_report(GT,preds))

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.87      0.92      0.89      1979
           1       0.92      0.87      0.90      2221

    accuracy                           0.90      4200
   macro avg       0.90      0.90      0.90      4200
weighted avg       0.90      0.90      0.90      4200



In [ ]:
trainer.save_model("classification_model_90_percent")

In [ ]:
#tokenizer.save_pretrained("classification_model_90_percent")

In [ ]:
pipe = pipeline("text-classification", model="classification_model_90_percent", tokenizer=tokenizer)
#pred= pipe("The product is perfect, I'll buy another one")
pred= pipe("The product beurkk i don't like it")
pred

Device set to use cuda:0


[{'label': 'LABEL_0', 'score': 0.9598471522331238}]

In [ ]:
pred_label = pred[0]['label']
if pred_label == 'LABEL_1':
  sentiment = 'positif'
else:
  sentiment = 'négatif'
print(f"La review est: {sentiment}")

La review est: négatif


In [ ]:
!zip -r classification_model_90_percent.zip /kaggle/working/classification_model_90_percent


  adding: kaggle/working/classification_model_90_percent/ (stored 0%)
  adding: kaggle/working/classification_model_90_percent/tokenizer_config.json (deflated 75%)
  adding: kaggle/working/classification_model_90_percent/training_args.bin (deflated 51%)
  adding: kaggle/working/classification_model_90_percent/model.safetensors (deflated 7%)
  adding: kaggle/working/classification_model_90_percent/config.json (deflated 49%)
  adding: kaggle/working/classification_model_90_percent/vocab.txt (deflated 53%)
  adding: kaggle/working/classification_model_90_percent/special_tokens_map.json (deflated 42%)
